In [1]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

# 1

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)


class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1,
                 n_hidden_neurons_2, n_out_neurons):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(n_in_neurons,n_hidden_neurons_1)
        self.act1 = nn.Tanh()

        self.fc2 = nn.Linear(n_hidden_neurons_1,n_hidden_neurons_2)
        self.act2 = nn.ReLU()

        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)
        self.act3 = nn.Sigmoid()

        nn.init.uniform_(self.fc1.weight, a=-2, b=2)
        nn.init.uniform_(self.fc1.bias, a=-2, b=2)
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.normal_(self.fc2.bias, mean=.5, std=.7)
        nn.init.xavier_normal_(self.fc3.weight)
        nn.init.normal_(self.fc3.bias, mean=.5, std=.7)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)

        x = self.fc2(x)
        x = self.act2(x)

        x = self.fc3(x)
        x = self.act3(x)
        return x

In [8]:
data = pd.read_csv('/home/stepbruh/Рабочий стол/Практикум/Специалист по Data Science Плюс. Часть 13 из 16 (2022)/Модели и алгоритмы в машинном обучении/04 Обучение нейронной сети/Electrical_Grid_Stability.csv', sep=';').drop('Unnamed: 0', axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=['stability']),
    data.stability,
    test_size=0.3,
    shuffle=True)

X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)


n_in_neurons = 12
n_hidden_neurons_1 = 8
n_hidden_neurons_2 = 4
n_out_neurons = 1

net = Net(n_in_neurons,n_hidden_neurons_1,n_hidden_neurons_2,n_out_neurons)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
loss = nn.BCELoss()
num_epochs = 100

In [12]:
for epoch in range(num_epochs):
    net.train()
    optimizer.zero_grad()
    preds = net.forward(X_train).flatten()
    loss_value = loss(preds, y_train)
    loss_value.backward()
    optimizer.step()

    if epoch%10 == 0 or epoch == num_epochs - 1:
        net.eval()
        test_preds = net.forward(X_test)
        accuracy = (torch.round(test_preds) == y_test).float().mean().data
        print(accuracy)

tensor(0.6371)
tensor(0.6360)
tensor(0.6341)
tensor(0.6317)
tensor(0.6306)
tensor(0.6292)
tensor(0.6287)
tensor(0.6279)
tensor(0.6269)
tensor(0.6265)
tensor(0.6258)


# 2

In [13]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

data = pd.read_csv('/home/stepbruh/Рабочий стол/Практикум/Специалист по Data Science Плюс. Часть 13 из 16 (2022)/Модели и алгоритмы в машинном обучении/04 Обучение нейронной сети/Electrical_Grid_Stability.csv', sep=';').drop('Unnamed: 0', axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=['stability']),
    data.stability,
    test_size=0.3,
    shuffle=True)

X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)

n_neurons = [12, 9, 6, 3, 1]
net_layers = []

for i in range(1, len(n_neurons) - 1):
		net_layers.append(nn.Linear(n_neurons[i-1], n_neurons[i]))
		if (i+1) % 2 == 0:
				net_layers.append(nn.Tanh())
		else:
				net_layers.append(nn.ReLU())

net_layers.append(nn.Linear(n_neurons[-2], n_neurons[-1]))
net_layers.append(nn.Sigmoid())

net = nn.Sequential(*net_layers) # такая запись позволяет передавать элементы списка как параметры для инициализации

def init_weights(layer):
    if type(layer) == nn.Linear: # Проверка, что слой – полносвязный
        nn.init.normal_(layer.weight, mean=.5, std=2)
        nn.init.normal_(layer.bias, mean=-.5, std=1)

net.apply(init_weights)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
loss = nn.BCELoss()
num_epochs = 100

In [14]:
for epoch in range(num_epochs):
    optimizer.zero_grad()

    preds = net.forward(X_train).flatten()

    loss_value = loss(preds, y_train)
    # print(loss_value)

    loss_value.backward()

    optimizer.step()

    if epoch % 10 == 0 or epoch == num_epochs - 1:
        net.eval()
        test_preds = net.forward(X_test)
        accuracy = (torch.round(test_preds) == y_test).float().mean().data
        print(accuracy)

tensor(0.5290)
tensor(0.5287)
tensor(0.5285)
tensor(0.5289)
tensor(0.5285)
tensor(0.5282)
tensor(0.5278)
tensor(0.5280)
tensor(0.5277)
tensor(0.5275)
tensor(0.5286)
